In [17]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/Jay Patel/OneDrive/Desktop/Datasets/mall_customers.csv")


In [18]:
print("Dataset Shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())
print("\nDataset Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())
print("\nDuplicate Rows:", df.duplicated().sum())

Dataset Shape: (200, 5)

First 5 rows:
   CustomerID  Gender  Age  Annual Income (k$)  Spending Score (1-100)
0           1    Male   19                  15                      39
1           2    Male   21                  15                      81
2           3  Female   20                  16                       6
3           4  Female   23                  16                      77
4           5  Female   31                  17                      40

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CustomerID              200 non-null    int64 
 1   Gender                  200 non-null    object
 2   Age                     200 non-null    int64 
 3   Annual Income (k$)      200 non-null    int64 
 4   Spending Score (1-100)  200 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 7.9+ KB
No

In [19]:
df_clean = df.copy()

In [20]:
df_clean.columns = df_clean.columns.str.lower().str.replace(' ', '_')
print("Standardized columns:", df_clean.columns.tolist())

Standardized columns: ['customerid', 'gender', 'age', 'annual_income_(k$)', 'spending_score_(1-100)']


In [21]:
numerical_cols = ['age', 'annual_income_(k$)', 'spending_score_(1-100)']
for col in numerical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

In [22]:
categorical_cols = ['gender']
for col in categorical_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].mode()[0])

In [23]:
initial_count = len(df_clean)
df_clean = df_clean.drop_duplicates()
duplicates_removed = initial_count - len(df_clean)

In [24]:
if 'gender' in df_clean.columns:
    df_clean['gender'] = df_clean['gender'].str.upper().str.strip()
    # Standardize gender values
    gender_mapping = {'F': 'FEMALE', 'M': 'MALE', 'FEMALE': 'FEMALE', 'MALE': 'MALE'}
    df_clean['gender'] = df_clean['gender'].map(gender_mapping).fillna(df_clean['gender'])

In [25]:
if 'customer_id' in df_clean.columns:
    df_clean['customer_id'] = df_clean['customer_id'].astype(str)

In [26]:
def handle_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df[column] = np.where(df[column] < lower_bound, lower_bound, df[column])
    df[column] = np.where(df[column] > upper_bound, upper_bound, df[column])
    return df

In [27]:
for col in numerical_cols:
    if col in df_clean.columns:
        df_clean = handle_outliers(df_clean, col)

In [28]:
if 'age' in df_clean.columns:
    bins = [0, 25, 35, 45, 55, 100]
    labels = ['18-25', '26-35', '36-45', '46-55', '55+']
    df_clean['age_group'] = pd.cut(df_clean['age'], bins=bins, labels=labels, right=False)

In [29]:
print("\n=== CLEANING SUMMARY ===")
print(f"Original dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Duplicate rows removed: {duplicates_removed}")
print(f"Missing values after cleaning:")
print(df_clean.isnull().sum())
print(f"\nData types after cleaning:")
print(df_clean.dtypes)
print(f"\nFirst 5 rows of cleaned data:")
print(df_clean.head())



=== CLEANING SUMMARY ===
Original dataset shape: (200, 5)
Cleaned dataset shape: (200, 6)
Duplicate rows removed: 0
Missing values after cleaning:
customerid                0
gender                    0
age                       0
annual_income_(k$)        0
spending_score_(1-100)    0
age_group                 0
dtype: int64

Data types after cleaning:
customerid                   int64
gender                      object
age                        float64
annual_income_(k$)         float64
spending_score_(1-100)     float64
age_group                 category
dtype: object

First 5 rows of cleaned data:
   customerid  gender   age  annual_income_(k$)  spending_score_(1-100)  \
0           1    MALE  19.0                15.0                    39.0   
1           2    MALE  21.0                15.0                    81.0   
2           3  FEMALE  20.0                16.0                     6.0   
3           4  FEMALE  23.0                16.0                    77.0   
4           5

In [30]:
df_clean.to_csv('mall_customers_cleaned.csv', index=False)

In [31]:
print(df_clean.describe())

       customerid         age  annual_income_(k$)  spending_score_(1-100)
count  200.000000  200.000000          200.000000              200.000000
mean   100.500000   38.850000           60.517500               50.200000
std     57.879185   13.969007           26.143551               25.823522
min      1.000000   18.000000           15.000000                1.000000
25%     50.750000   28.750000           41.500000               34.750000
50%    100.500000   36.000000           61.500000               50.000000
75%    150.250000   49.000000           78.000000               73.000000
max    200.000000   70.000000          132.750000               99.000000
